In [2]:
# from utils.HWGQQuantizer import HWGQQuantizer
# import utils
import torch

from HWGQQuantizer import HWGQQuantizer
weights = torch.rand(2,3,4)
# print(weights)
modle = HWGQQuantizer(bit_width=4)
print(modle.quantize(weights))

TypeError: linspace() received an invalid combination of arguments - got (int, int, float), but expected one of:
 * (Tensor start, Tensor end, int steps, *, Tensor out = None, torch.dtype dtype = None, torch.layout layout = None, torch.device device = None, bool pin_memory = False, bool requires_grad = False)
 * (Number start, Tensor end, int steps, *, Tensor out = None, torch.dtype dtype = None, torch.layout layout = None, torch.device device = None, bool pin_memory = False, bool requires_grad = False)
 * (Tensor start, Number end, int steps, *, Tensor out = None, torch.dtype dtype = None, torch.layout layout = None, torch.device device = None, bool pin_memory = False, bool requires_grad = False)
 * (Number start, Number end, int steps, *, Tensor out = None, torch.dtype dtype = None, torch.layout layout = None, torch.device device = None, bool pin_memory = False, bool requires_grad = False)


In [3]:
from utils.layers import *

student_output = torch.tensor([[2.0, 0.5, 1.0], [1.5, 1.2, 2.2]], requires_grad=True)
combined_teacher_output = torch.tensor([[2.1, 0.4, 1.1], [1.6, 1.1, 2.3]], requires_grad=True)
labels = torch.tensor([0, 2])  
temperature = 2.0 

loss = loss_kl_divergence_with_logits(student_output, labels, combined_teacher_output, temperature)

print("KL Divergence Loss with logits:", loss.item())

KL Divergence Loss with logits: -3.787278652191162


In [16]:
def forward(x, bit_width):
        return x.clamp(-1, 1).mul(2**bit_width).round() / (2**bit_width)

In [80]:
def forward99(x, bit_width):
    return 2*(torch.floor(((2**bit_width-1)/2)*(x+1))/(2**bit_width-1) - 0.5)

In [ ]:
def quantize(x, bit_width):
    sigma = x.std()
    q_levels = sigma * torch.linspace(0, 3, 2**bit_width)
    # print(q_levels)
    return q_levels[((x / sigma).unsqueeze(-1) >= q_levels[:-1]) & ((x / sigma).unsqueeze(-1) < q_levels[1:])].max(dim=-1).values

In [ ]:
def forward2(x, bit_width):
    sigma = x.std()  # Standard deviation for scaling
    return (sigma * torch.bucketize(x / sigma, torch.linspace(0, 3, 2**bit_width, device=x.device) , right=True) / (2**bit_width - 1)) * sigma


In [63]:
def forward3(x, bit_width):
    sigma = x.std()  # Standard deviation for scaling
    q_levels = sigma*torch.linspace(0, 3, 2**bit_width, device=x.device)  # Quantization levels
    # print(sigma)
    return q_levels[torch.bucketize(x.flatten(), q_levels, right=True) - 1].view_as(x)


In [39]:
def forward_new_inline(x, bit_width):
    sigma = x.std()
    q_levels = torch.linspace(0, 3, 2**bit_width)
    q_levels = q_levels.view(1, -1).expand_as(x)
    # print()
    # print(x >= (sigma * q_levels[:-1]))
    # print(sigma * q_levels[1:])
    # print(sigma * q_levels[:-1])
    return torch.where(x >= sigma * q_levels[:-1], sigma * q_levels[1:], sigma * q_levels[:-1])

In [35]:
def forward_new(x, bit_width):
    sigma = x.std()  
    q_levels = torch.linspace(0, 3, 2**bit_width) 
    # print("q.levels ki shape", q_levels.shape)
    quantized_x = torch.zeros_like(x)
    for i, q in enumerate(q_levels):
        mask = (x >= (sigma * q_levels[i - 1] if i > 0 else 0)) & (x < sigma * q)
        quantized_x[mask] = sigma * q

    return quantized_x

In [81]:
import torch
x = torch.rand(3,4) * 2 - 1  # Random input in range [-1, 1]
bit_width = 4

# Compute outputs
print(x)
out2 = forward_new(x, bit_width)
out3 = forward99(x, bit_width)

# out1 = forward_new_inline(x, bit_width)
# print(out1)
print(out2)
print(out3)
# Compare outputs
try:
    torch.testing.assert_close(out3, out2, rtol=1e-4, atol=1e-4)
    print("Test passed: Outputs are the same.")
except AssertionError as e:
    print("Test failed: Outputs are different.")
    print(e)

tensor([[-0.0307,  0.7787, -0.6081,  0.3215],
        [ 0.3956, -0.0367, -0.2510,  0.2963],
        [-0.9986,  0.8470,  0.5298, -0.6073]])
tensor([[0.0000, 0.8141, 0.0000, 0.3489],
        [0.4652, 0.0000, 0.0000, 0.3489],
        [0.0000, 0.9304, 0.5815, 0.0000]])
tensor([[-0.0667,  0.7333, -0.7333,  0.2000],
        [ 0.3333, -0.0667, -0.3333,  0.2000],
        [-1.0000,  0.7333,  0.4667, -0.7333]])
Test failed: Outputs are different.
Tensor-likes are not close!

Mismatched elements: 12 / 12 (100.0%)
Greatest absolute difference: 1.0 at index (2, 0) (up to 0.0001 allowed)
Greatest relative difference: inf at index (0, 0) (up to 0.0001 allowed)


In [60]:
x_final 

tensor([[ 0.3348, -0.9144,  0.6856, -0.9406],
        [-0.0540, -0.0649,  0.8615, -0.2454],
        [-0.4563, -0.3982,  0.9035,  0.0547]])